# 在网页中应用实时MNIST识别
> 小惊喜中带点美中不足吧

- toc: true
- badges: true
- comments: true
- hide: true
- categories: [tensorflow, gradio]

## 前言

偶然发现一个包[Gradio](www.gradio.app)，可以快速构建基于机器学习应用的Web交互接口。这里尝试能否使用在Fastpages的Jupyter Notebook文章中。

这里直接应用官网的demo之一——实时识别手写数字。

## 上手

1. 先训练一个MNIST分类器。

In [1]:
import tensorflow as tf
import gradio as gr

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32').reshape(-1, 28, 28, 1) / 255.
x_test = x_test.astype('float32').reshape(-1, 28, 28, 1) / 255.

# Lenet 5
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=6, kernel_size=5, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(pool_size=2),
    
    tf.keras.layers.Conv2D(filters=16, kernel_size=5, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2),
    
    tf.keras.layers.Conv2D(filters=120, kernel_size=5, activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(84, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, verbose=0)

11493376/11490434 [==============================] - 5s 0us/step


2. 构建Gradio界面

In [2]:
# gradio interface need 3 parameters：predict function、input & output components

# predict function
def classify(image):
    image = image.reshape(-1, 28, 28, 1)
    prediction = model.predict(image).tolist()[0]
    return {str(i): prediction[i] for i in range(10)}

# input component
sketchpad = gr.inputs.Sketchpad()

# output components
label = gr.outputs.Label(num_top_classes=3)

# generate the UI
# live=True: reload changes automatically
# capture_sessiong=True: specifically for tensorflow 1.0
interface = gr.Interface(classify, sketchpad, label, live=True, capture_session=True)
interface.launch(share=True)

Running locally at: http://127.0.0.1:7860/
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://44307.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://44307.gradio.app')

## 结果

功能都正常实现出来了，虽然识别准确率感人，但关我们主角Gradio什么事。

但是Gradio文档展现的api确实有点少，比如本例中UI显示范围受到限制，想要调整UI大小却找不到相应的参数。总的来说Gradio更偏向傻瓜式，应用起来确实快捷简单。

* 优点：

    + 方便快捷

* 缺点：

    - api较少，可定义幅度小